***Step 1***

In [2]:
#Extract structured lines from the PDF
import pdfplumber
pdf_path = "combined 21.pdf"
all_lines = []
with pdfplumber.open(pdf_path) as pdf:
   for page_num, page in enumerate(pdf.pages, start=1):
       text = page.extract_text()
       if text:
           lines = [line.strip() for line in text.split('\n') if line.strip()]
           all_lines.extend(lines)
# Confirm extraction
print(f" Total lines fetched from PDF: {len(all_lines)}")
# Preview first 4 records (each record = 3 lines)
print("\n Sample 4 Records (12 lines):")
print("\n".join(all_lines[:12]))




Cannot set gray non-stroke color because /'P2' is an invalid float value
Cannot set gray stroke color because /'P0' is an invalid float value


***Step 2***

In [1]:
import re
import pandas as pd


people = {}
# Loop over all lines (without fixed 3-line steps)
for i in range(len(all_lines)):
   line = all_lines[i]
   # Check if the line contains core data
   if 'CPN' in line and 'HHID' in line and 'NAME' in all_lines[i+1] and 'ADDR' in all_lines[i+2]:
       creative = re.search(r'CV\s+(\S+)', line)
       version = re.search(r'PG(\d+)', line)
       cpn = re.search(r'CPN\s+(\d+)', line)
       hhid = re.search(r'HHID\s+(\d+)', line)
       name = re.search(r'NAME\s+(.+)', all_lines[i+1])
       addr = re.search(r'ADDR\s+(.+)', all_lines[i+2])
       if hhid:
           key = hhid.group(1)
       elif name:
           key = name.group(1).strip()
       else:
           continue
       if key not in people:
           people[key] = {
               'name': name.group(1).strip() if name else '',
               'household_id': hhid.group(1) if hhid else '',
               'address': addr.group(1).strip() if addr else '',
               'creative': creative.group(1) if creative else '',
               'version': f"PG{version.group(1)}" if version else '',
               'slots': []
           }
       if cpn:
           people[key]['slots'].append(cpn.group(1))
# Convert to DataFrame
rows = []
for person in people.values():
   row = {
       'full_name': person['name'],
       'household_id': person['household_id'],
       'full_address': person['address'],
       'creative_version': person['creative']
      # 'version': person['version']
   }
   for idx, slot in enumerate(person['slots']):
       row[f'slot{idx+1}'] = slot
   rows.append(row)
df = pd.DataFrame(rows)
df

NameError: name 'all_lines' is not defined

***Step 3***

In [5]:
import re
import pandas as pd

people = {}

# Loop over all lines (without fixed 3-line steps)
for i in range(len(all_lines)):
    line = all_lines[i]

    # Check if the line contains core data
    if 'CPN' in line and 'HHID' in line and 'NAME' in all_lines[i+1] and 'ADDR' in all_lines[i+2]:
        creative = re.search(r'CV\s+(\S+)', line)
        version = re.search(r'PG(\d+)', line)
        cpn = re.search(r'CPN\s+(\d+)', line)
        hhid = re.search(r'HHID\s+(\d+)', line)
        name = re.search(r'NAME\s+(.+)', all_lines[i+1])
        addr = re.search(r'ADDR\s+(.+)', all_lines[i+2])

        # Join address with comma if next line is city/state/ZIP
        #address = addr.group(1).strip() if addr else ''
        #if i + 3 < len(all_lines):
         #   next_line = all_lines[i + 3].strip()
          #  if re.search(r'\b[A-Z]{2} \d{5}(-\d{4})?\b', next_line):
           #     address += ', ' + next_line

        
        # Join address with comma if next line is city/state/ZIP
        address = addr.group(1).strip() if addr else ''
        if i + 3 < len(all_lines):
                next_line = all_lines[i + 3].strip()
                if re.search(r'\b[A-Z]{2} \d{5}(-\d{4})?\b', next_line):
                    address += ', ' + next_line  # Ensure comma is added

        # Use HHID or name as key
        if hhid:
            key = hhid.group(1)
        elif name:
            key = name.group(1).strip()
        else:
            continue

        # Store data
        if key not in people:
            people[key] = {
                'name': name.group(1).strip() if name else '',
                'household_id': hhid.group(1) if hhid else '',
                'address': address,
                'creative': creative.group(1) if creative else '',
                'version': f"PG{version.group(1)}" if version else '',
                'slots': []
            }

        if cpn:
            people[key]['slots'].append(cpn.group(1))

# Convert to DataFrame
rows = []
for person in people.values():
    row = {
        'full_name': person['name'],
        'household_id': person['household_id'],
        'full_address': person['address'],
        'creative_version': person['creative']
        # 'version': person['version']
    }
    for idx, slot in enumerate(person['slots']):
        row[f'slot{idx+1}'] = slot
    rows.append(row)

df = pd.DataFrame(rows)
df.to_excel("final_output_MM.xlsx")



***Step 4***

In [6]:
import pandas as pd

#loading excel file with multiple headers

output_excel = pd.read_excel("MM 2505_Automation_Output.xls",header=[0,1],engine='xlrd')
output_excel.reset_index(drop=True,inplace=True)


#defining base fields
base_fields=[
    ("full_name","Audit"),
    ("full_name","OCR"),
    ("household_id","Audit"),
    ("household_id","OCR"),
    ("full_address","Audit"),
    ("full_address","OCR"),
    ("creative_version","Audit"),
    ("creative_version","OCR")
]

slot_fields =[]
for i in range(1,17):
    slot = f"slot {i}"
    slot_fields.extend([
    (slot,"Audit"),
    (slot,"Back Page OCR"),
    (slot,"Front Page OCR")
    ])

all_fields = base_fields + slot_fields





***Step 5***

In [7]:
test=all_fields
test=pd.DataFrame(all_fields)
test

,0,1
0,full_name,Audit
1,full_name,OCR
2,household_id,Audit
3,household_id,OCR
4,full_address,Audit
5,full_address,OCR
6,creative_version,Audit
7,creative_version,OCR
8,slot 1,Audit
9,slot 1,Back Page OCR


***Step 6***

In [8]:
output_excel = pd.read_excel("MM 2505_Automation_Output.xls",header=[0,1],engine='xlrd')
df_ex = pd.DataFrame(output_excel)
df_ex

full_name                   household_id               \
               Audit               OCR        Audit          OCR   
0      LINDA KENNEDY     LINDA KENNEDY    105808808    105808808   
1       MARGO MATHER      MARGO MATHER    118735088    118735088   
2   STEPHANIE HOWARD  STEPHANIE HOWARD    194087214    194087214   
3     LEANNE SHEDDEN    LEANNE SHEDDEN    326244238    326244238   
4      TERESA BRAZIL    TERESA BRAZILL    128090120    128090120   
..               ...               ...          ...          ...   
58       BETTY SMITH         BETY SMIT    268082193    268082193   
59       NORA KENNEY       NORA WENNEY    287251976    287251976   
60     TAMRA GARDNER     TAMRA GARDNER  35003000268  35003000268   
61     JULIE CORALLO    JULIE CORALLLO    182426486    182426486   
62    KENNY WILLIAMS     KENNY WLLLAMS      3839360      3839360   

                                      full_address  \
                                             Audit   
0        303 THE ALAMEDA MIDDLETOWN, OH 45044-4807   
1         3384 ALEXIS RD CINCINNATI, OH 45239-4015   
2       2472 STATE ROUTE 131 GOSHEN, OH 45122-9412   
3      122 W ARROWWOOD RD OAK RIDGE, TN 37830-5705   
4      4077 BLUE LEVEL RD ROCKFIELD, KY 42274-9351   
..                                             ...   
58    7111 MEADOWBROOK RD LYNCHBURG, VA 24502-2005   
59     320 WARREN ST CHRISTIANSBURG, VA 24073-1841   
60  11580 FURY LN UNIT 165 EL CAJON, CA 92019-4348   
61     210 N TROTWOOD AVE SAN PEDRO, CA 90732-2732   
62     1838 S ORANGE DR LOS ANGELES, CA 90019-5319   

                                                  creative_version           \
                                              OCR            Audit      OCR   
0        303 THE ALAMEDA MIDDLETOWN OH 45044-4807          M53MK1A  M53MK1A   
1         3384 ALEXIS RD CINCINNATI OH 45239-4015          M53MK1A  M53MK1A   
2       2472 STATE ROUTE 131 GOSHEN OH 45122-9412          M53MK1A  M53MK1A   
3       122 W ARROWWOOD RD OAKRIDGE TN 37830-5705          M53MK1A  M53MI1A   
4      4077 BLUE LEVEL RD ROCKFIELD IY 42274-9351          M53MK1A  M53MI1A   
..                                            ...              ...      ...   
58    7111 MEADOWBROOK RD LYNCHBURG VA 24502-2005          M53MK1A  M53MI1A   
59     320 WARREN ST CHRISTIANSBURG VA 24073-1841          M53MK1A  M53MI1A   
60  11580 FURY LN UNIT 165 EL CAUON CA 92019-4348          M53MR05  M53MR05   
61     210 N TROTWOOD AVE SAN PEDRO CA 90732-2732          M53MR05  M53MR05   
62     1838 S ORANGE DR LOS ANGELES CA 90019-5319          M53MR05  M53MR05   

   slot 1        ... slot 12       slot 13       slot 14       slot 15        \
    Audit   OCR  ...   Audit   OCR   Audit   OCR   Audit   OCR   Audit   OCR   
0    4441  4441  ...    3134  3134    2307  2307    5542  5542    2312  2312   
1    4192  4192  ...    4123  4123    4382  4382    5810  5810    4191  4191   
2    5103  5103  ...    2190  2190    2165  2165    5547  5547    2038  2038   
3    4421  4421  ...    2184  2184    2024  2024    5542  5542    2038  2038   
4    4507  4507  ...    4684  4684    5244  5244    4401  4401    2313  2313   
..    ...   ...  ...     ...   ...     ...   ...     ...   ...     ...   ...   
58   5542  5542  ...    4136  4136    2192  2192    3103  3103    2236  2236   
59   4495  4495  ...    4136  4136    4312  4312    2266  2266    2339  2339   
60   4495  4495  ...    4448  4448    4884  4884    4268  4268    4135  4135   
61   4495  4495  ...    5076  5076    4448  4448    4192  4192    4766  4766   
62   4495  4495  ...    2005  2005    2280  2280    2035  2035    5540  5540   

   slot 16        
     Audit   OCR  
0     3033  3033  
1     3136  3136  
2     2116  2116  
3     2317  2317  
4     2093  2093  
..     ...   ...  
58    3109  3109  
59    3030  3030  
60    2221  2221  
61    2339  2339  
62    2180  2180  

[63 rows x 40 columns]

***Step 7***

In [9]:

import pandas as pd
from difflib import get_close_matches

# Load the audit and OCR output files
df_audit = pd.read_excel("MM 2505_Automation_Output.xls", sheet_name="Matches", header=[0, 1], engine="xlrd")
df_ocr = pd.read_excel("final_output_MM.xlsx", engine="openpyxl")

# Flatten multi-level headers in audit file
df_audit.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in df_audit.columns]

# Rename audit columns for easier access
df_audit = df_audit.rename(columns={
    'full_name_Audit': 'full_name',
    'household_id_Audit': 'household_id',
    'full_address_Audit': 'full_address',
    'creative_version_Audit': 'creative_version'
})

# Clean full_name in both DataFrames
df_ocr['full_name'] = df_ocr['full_name'].astype(str).str.strip().str.upper()
df_audit['full_name'] = df_audit['full_name'].astype(str).str.strip().str.upper()

# Helper functions
def normalize_id(val):
    try:
        return str(int(float(val)))
    except:
        return str(val).strip()

def is_missing(val):
    val = str(val).strip().upper()
    return pd.isna(val) or val in ["MISSING", "", "NAN", "-", "- - -"]

# Apply fuzzy matching to map OCR names to audit names
audit_names = df_audit['full_name'].dropna().unique().tolist()
df_ocr['matched_name'] = df_ocr['full_name'].apply(lambda x: get_close_matches(x, audit_names, n=1, cutoff=0.85)[0] if get_close_matches(x, audit_names, n=1, cutoff=0.85) else None)

# Set index for mapping
df_map = df_ocr.set_index('matched_name')

# Define columns to compare and update
columns = ['full_name', 'household_id', 'full_address', 'creative_version']
subcols = ['Audit', 'OCR']

# Create a copy for updates
df_audit_updated = df_audit.copy()
for idx in df_audit.index:
    name = df_audit.at[idx, 'full_name'].strip().upper()
    if name in df_map.index:
        for col in columns:
            if col in df_map.columns:
                correct_val = str(df_map.at[name, col]).strip()
                for subcol in subcols:
                    key = f"{col}_{subcol}"
                    if key in df_audit.columns:
                        current_val = str(df_audit.at[idx, key]).strip().upper()
                        if col == 'household_id':
                            correct_val = normalize_id(correct_val)
                            current_val = normalize_id(current_val)
                        if is_missing(current_val) or current_val != correct_val.upper():
                            df_audit_updated.at[idx, key] = correct_val

# Save the updated Excel file
df_audit_updated.to_excel("final_output_MM_fully_corrected.xlsx", index=False)


C:\Users\t196876\AppData\Local\Temp\ipykernel_17548\3067350072.py:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '57722873' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_audit_updated.at[idx, key] = correct_val
